<a href="https://colab.research.google.com/github/Yashasvi281003/EE-533---CUDA-Lab/blob/main/MatrixMul_CUDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%writefile hello_world.cu
#include <stdio.h>
__global__ void hello() {
    printf("Hello World\n");
}

int main () {
    printf("Before GPU\n");
    hello<<<1,1>>>();
    cudaDeviceReset();
    printf("After GPU\n");
    return 0;
}

Overwriting hello_world.cu


In [ ]:
!nvcc hello_world.cu -o hello_world
!./hello_world

Before GPU
After GPU


In [ ]:
%%writefile sizetest.c
#include <stdio.h>
int main(){
  printf("Size of system = %zu", sizeof(int));
  return 0;
}


Overwriting sizetest.c


In [ ]:
!gcc sizetest.c -o sizetest
!./sizetest

Size of system = 4

In [ ]:
%%writefile matrix_gpu.cu
#include <stdio.h>
#include <stdlib.h>
#include <cuda_runtime.h>

__global__ void matrixMultiplyGPU(float *A, float *B, float *C, int N) {
 int row = blockIdx.y * blockDim.y + threadIdx.y;
 int col = blockIdx.x * blockDim.x + threadIdx.x;
 if (row < N && col < N) {
 float sum = 0.0f;
 for (int k = 0; k < N; k++) {
 sum += A[row * N + k] * B[k * N + col];
 }
 C[row * N + col] = sum;
 }
}

int main(int argc, char **argv) {
    int N = (argc > 1) ? atoi(argv[1]) : 512;
    size_t size = N * N * sizeof(float);

    float *h_A = (float *)malloc(size);
    float *h_B = (float *)malloc(size);
    float *h_C = (float *)malloc(size);

    for (int i = 0; i < N * N; i++) {
        h_A[i] = rand() / (float)RAND_MAX;
        h_B[i] = rand() / (float)RAND_MAX;
    }

    float *d_A, *d_B, *d_C;
    cudaMalloc((void**)&d_A, size);
    cudaMalloc((void**)&d_B, size);
    cudaMalloc((void**)&d_C, size);
    cudaMemcpy(d_A, h_A, size, cudaMemcpyHostToDevice);
    cudaMemcpy(d_B, h_B, size, cudaMemcpyHostToDevice);

    dim3 dimBlock(16, 16);
    dim3 dimGrid((N + 15) / 16, (N + 15) / 16);

    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);
    cudaEventRecord(start);
    matrixMultiplyGPU<<<dimGrid, dimBlock>>>(d_A, d_B, d_C, N);
    cudaEventRecord(stop);
    cudaDeviceSynchronize();
    float ms = 0.0f;
    cudaEventElapsedTime(&ms, start, stop);
    cudaMemcpy(h_C, d_C, size, cudaMemcpyDeviceToHost);
    printf("Naive CUDA execution time (N=%d): %.3f ms\n", N, ms);
    free(h_A); free(h_B); free(h_C);
    cudaFree(d_A); cudaFree(d_B); cudaFree(d_C);
    return 0;
}


Overwriting matrix_gpu.cu


In [ ]:
%%bash
nvcc matrix_gpu.cu -o matrix_gpu
for N in 8 16 32 64 128 256 400 512 562 1024 1700 1962 2048
do
  ./matrix_gpu $N
done

Naive CUDA execution time (N=8): 10.337 ms
Naive CUDA execution time (N=16): 11.295 ms
Naive CUDA execution time (N=32): 10.887 ms
Naive CUDA execution time (N=64): 10.709 ms
Naive CUDA execution time (N=128): 10.934 ms
Naive CUDA execution time (N=256): 10.893 ms
Naive CUDA execution time (N=400): 10.819 ms
Naive CUDA execution time (N=512): 10.580 ms
Naive CUDA execution time (N=562): 12.400 ms
Naive CUDA execution time (N=1024): 10.426 ms
Naive CUDA execution time (N=1700): 7.153 ms
Naive CUDA execution time (N=1962): 7.152 ms
Naive CUDA execution time (N=2048): 7.351 ms


In [ ]:
%%writefile matrix_smt.cu
#include <stdio.h>
#include <stdlib.h>
#include <cuda_runtime.h>

#define TILE_WIDTH 16

__global__ void matrixMultiplyTiled(float *A, float *B, float *C, int N) {
    __shared__ float ds_A[TILE_WIDTH][TILE_WIDTH];
    __shared__ float ds_B[TILE_WIDTH][TILE_WIDTH];

    int bx = blockIdx.x;
    int by = blockIdx.y;
    int tx = threadIdx.x;
    int ty = threadIdx.y;

    int Row = by * TILE_WIDTH + ty;
    int Col = bx * TILE_WIDTH + tx;

    float Pvalue = 0.0f;

    for (int m = 0; m < (N + TILE_WIDTH - 1) / TILE_WIDTH; ++m) {
        if (Row < N && m*TILE_WIDTH + tx < N)
            ds_A[ty][tx] = A[Row * N + m*TILE_WIDTH + tx];
        else
            ds_A[ty][tx] = 0.0f;

        if (Col < N && m*TILE_WIDTH + ty < N)
            ds_B[ty][tx] = B[(m*TILE_WIDTH + ty) * N + Col];
        else
            ds_B[ty][tx] = 0.0f;

        __syncthreads();

        for (int k = 0; k < TILE_WIDTH; ++k)
            Pvalue += ds_A[ty][k] * ds_B[k][tx];

        __syncthreads();
    }

    if (Row < N && Col < N)
        C[Row * N + Col] = Pvalue;
}

int main(int argc, char **argv) {
    int N = (argc > 1) ? atoi(argv[1]) : 512;
    size_t size = N * N * sizeof(float);

    float *h_A = (float *)malloc(size);
    float *h_B = (float *)malloc(size);
    float *h_C = (float *)malloc(size);

    for (int i = 0; i < N * N; i++) {
        h_A[i] = rand() / (float)RAND_MAX;
        h_B[i] = rand() / (float)RAND_MAX;
    }

    float *d_A, *d_B, *d_C;
    cudaMalloc((void**)&d_A, size);
    cudaMalloc((void**)&d_B, size);
    cudaMalloc((void**)&d_C, size);

    cudaMemcpy(d_A, h_A, size, cudaMemcpyHostToDevice);
    cudaMemcpy(d_B, h_B, size, cudaMemcpyHostToDevice);

    dim3 dimBlock(TILE_WIDTH, TILE_WIDTH);
    dim3 dimGrid((N + TILE_WIDTH - 1) / TILE_WIDTH,
                 (N + TILE_WIDTH - 1) / TILE_WIDTH);

    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);

    cudaEventRecord(start);
    matrixMultiplyTiled<<<dimGrid, dimBlock>>>(d_A, d_B, d_C, N);
    cudaEventRecord(stop);
    cudaDeviceSynchronize();

    float ms;
    cudaEventElapsedTime(&ms, start, stop);

    printf("Optimized CUDA time (N=%d): %.3f ms\n", N, ms);

    free(h_A); free(h_B); free(h_C);
    cudaFree(d_A); cudaFree(d_B); cudaFree(d_C);
    return 0;
}


Writing matrix_smt.cu


In [ ]:
%%bash
nvcc matrix_smt.cu -o matrix_smt
for N in 8 16 32 64 128 256 400 512 562 1024 1700 1962 2048
do
  ./matrix_smt $N
done

Optimized CUDA time (N=8): 7.472 ms
Optimized CUDA time (N=16): 7.535 ms
Optimized CUDA time (N=32): 7.352 ms
Optimized CUDA time (N=64): 7.387 ms
Optimized CUDA time (N=128): 7.341 ms
Optimized CUDA time (N=256): 7.243 ms
Optimized CUDA time (N=400): 7.353 ms
Optimized CUDA time (N=512): 7.382 ms
Optimized CUDA time (N=562): 7.152 ms
Optimized CUDA time (N=1024): 7.182 ms
Optimized CUDA time (N=1700): 7.380 ms
Optimized CUDA time (N=1962): 7.892 ms
Optimized CUDA time (N=2048): 7.723 ms


In [ ]:
%%writefile matrix_cublas.cu
#include <stdio.h>
#include <stdlib.h>
#include <cuda_runtime.h>
#include <cublas_v2.h>

int main(int argc, char **argv) {
    int N = (argc > 1) ? atoi(argv[1]) : 512;
    size_t size = N * N * sizeof(float);

    float *h_A = (float *)malloc(size);
    float *h_B = (float *)malloc(size);
    float *h_C = (float *)malloc(size);

    for (int i = 0; i < N * N; i++) {
        h_A[i] = rand() / (float)RAND_MAX;
        h_B[i] = rand() / (float)RAND_MAX;
    }

    float *d_A, *d_B, *d_C;
    cudaMalloc((void**)&d_A, size);
    cudaMalloc((void**)&d_B, size);
    cudaMalloc((void**)&d_C, size);

    cudaMemcpy(d_A, h_A, size, cudaMemcpyHostToDevice);
    cudaMemcpy(d_B, h_B, size, cudaMemcpyHostToDevice);

    cublasHandle_t handle;
    cublasCreate(&handle);

    float alpha = 1.0f, beta = 0.0f;

    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);

    cudaEventRecord(start);
    cublasSgemm(handle,
                CUBLAS_OP_N, CUBLAS_OP_N,
                N, N, N,
                &alpha,
                d_B, N,
                d_A, N,
                &beta,
                d_C, N);
    cudaEventRecord(stop);
    cudaDeviceSynchronize();

    float ms;
    cudaEventElapsedTime(&ms, start, stop);

    printf("cuBLAS time (N=%d): %.3f ms\n", N, ms);

    cublasDestroy(handle);
    cudaFree(d_A); cudaFree(d_B); cudaFree(d_C);
    free(h_A); free(h_B); free(h_C);
    return 0;
}


Writing matrix_cublas.cu


In [ ]:
%%bash
nvcc matrix_cublas.cu -o matrix_cublas -lcublas
for N in 8 16 32 64 128 256 400 512 562 1024 1700 1962 2048
do
  ./matrix_cublas $N
done

cuBLAS time (N=8): 48.842 ms
cuBLAS time (N=16): 5.491 ms
cuBLAS time (N=32): 9.968 ms
cuBLAS time (N=64): 5.226 ms
cuBLAS time (N=128): 7.275 ms
cuBLAS time (N=256): 23.923 ms
cuBLAS time (N=400): 7.315 ms
cuBLAS time (N=512): 5.387 ms
cuBLAS time (N=562): 6.127 ms
cuBLAS time (N=1024): 6.416 ms
cuBLAS time (N=1700): 9.249 ms
cuBLAS time (N=1962): 11.226 ms
cuBLAS time (N=2048): 11.610 ms


In [ ]:
%%writefile cuda_lib.cu
#include <cuda_runtime.h>
#include <stdio.h>
#define TILE_WIDTH 16
__global__ void matrixMultiplyTiled(float *A, float *B, float *C, int N) {
 __shared__ float ds_A[TILE_WIDTH][TILE_WIDTH];
 __shared__ float ds_B[TILE_WIDTH][TILE_WIDTH];
 int bx = blockIdx.x; int by = blockIdx.y;
 int tx = threadIdx.x; int ty = threadIdx.y;
 int Row = by * TILE_WIDTH + ty;
 int Col = bx * TILE_WIDTH + tx;
 float Pvalue = 0.0;
 for (int m = 0; m < (N + TILE_WIDTH - 1) / TILE_WIDTH; ++m) {
 if (Row < N && (m*TILE_WIDTH+tx) < N)
 ds_A[ty][tx] = A[Row * N + m * TILE_WIDTH + tx];
 else
 ds_A[ty][tx] = 0.0f;
 if (Col < N && (m*TILE_WIDTH+ty) < N)
 ds_B[ty][tx] = B[(m*TILE_WIDTH + ty) * N + Col];
 else
 ds_B[ty][tx] = 0.0f;
 __syncthreads();
 for (int k = 0; k < TILE_WIDTH; ++k)
 Pvalue += ds_A[ty][k] * ds_B[k][tx];
 __syncthreads();
 }
 if (Row < N && Col < N)
 C[Row * N + Col] = Pvalue;
}
// Exposed C function for Python
extern "C" void gpu_matrix_multiply(float *h_A, float *h_B, float *h_C, int
N) {
 size_t size = N * N * sizeof(float);
 float *d_A, *d_B, *d_C;
 cudaMalloc((void**)&d_A, size);
 cudaMalloc((void**)&d_B, size);
 cudaMalloc((void**)&d_C, size);
 cudaMemcpy(d_A, h_A, size, cudaMemcpyHostToDevice);
 cudaMemcpy(d_B, h_B, size, cudaMemcpyHostToDevice);
 dim3 dimBlock(TILE_WIDTH, TILE_WIDTH);
 dim3 dimGrid((N + TILE_WIDTH - 1) / TILE_WIDTH, (N + TILE_WIDTH - 1) /
TILE_WIDTH);
 matrixMultiplyTiled<<<dimGrid, dimBlock>>>(d_A, d_B, d_C, N);
 cudaDeviceSynchronize();
 cudaMemcpy(h_C, d_C, size, cudaMemcpyDeviceToHost);
 cudaFree(d_A); cudaFree(d_B); cudaFree(d_C);
}

Overwriting cuda_lib.cu


In [ ]:
!nvcc -Xcompiler -fPIC -shared cuda_lib.cu -o libmatrix.so

In [48]:
import ctypes
import numpy as np
import time
# Load shared library
lib = ctypes.cdll.LoadLibrary("./libmatrix.so")
# Define argument types
lib.gpu_matrix_multiply.argtypes = [
 np.ctypeslib.ndpointer(dtype=np.float32, ndim=1, flags="C_CONTIGUOUS"),
 np.ctypeslib.ndpointer(dtype=np.float32, ndim=1, flags="C_CONTIGUOUS"),
 np.ctypeslib.ndpointer(dtype=np.float32, ndim=1, flags="C_CONTIGUOUS"),
 ctypes.c_int
]

N_values = [8, 16, 32, 64, 128, 256, 512, 1024, 2048]

for N in N_values:
    A = np.random.rand(N, N).astype(np.float32)
    B = np.random.rand(N, N).astype(np.float32)
    C = np.zeros((N, N), dtype=np.float32)

    start = time.time()
    lib.gpu_matrix_multiply(A.ravel(), B.ravel(), C.ravel(), N)
    end = time.time()
    print(f"Python call to CUDA library for N={N} completed in {end - start:.4f} seconds")

Python call to CUDA library for N=8 completed in 0.0012 seconds
Python call to CUDA library for N=16 completed in 0.0004 seconds
Python call to CUDA library for N=32 completed in 0.0004 seconds
Python call to CUDA library for N=64 completed in 0.0005 seconds
Python call to CUDA library for N=128 completed in 0.0006 seconds
Python call to CUDA library for N=256 completed in 0.0005 seconds
Python call to CUDA library for N=512 completed in 0.0015 seconds
Python call to CUDA library for N=1024 completed in 0.0038 seconds
Python call to CUDA library for N=2048 completed in 0.0208 seconds
